In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path

# directory : 

home = str(Path.home())
myPath = os.path.join(home, "Dropbox", "MobRes")
os.chdir(myPath)


In [2]:
columns_intrest = set()

In [3]:

# Sample all the datasets
for i in range(2013,2020):
    if i>2015:
        ext='csv'
    elif i<=2015:
        ext='txt'
    df = pd.read_csv(f"./raw_data/mig/census/FD_MIGCOM_{i}.{ext}", sep=';',low_memory=False, dtype='str', nrows=5000000)
    df['year'] = i
    df['year']=df['year'].astype(str)
    for name in df.columns:
        columns_intrest.add(name)
    
    ### Here we keep population we are intrested in
    
    # Lien à la personne de référence du ménage : Need to keep only main and partner (1 & 2)
    df = df[(df.LPRM=="1")|(df.LPRM=="2")]
    # Type d'activité # Need to take away job seekers, pensioners & students 
    df = df[df.TACT=="11"]
    # Type de logement # Keep only houses and flats
    df = df[(df.TYPL=="1")|(df.TYPL=="2")]

    # Keep only if person lived in the metropal the year before (indicators<6)
    df = df[(df.IRAN=="1")|(df.IRAN=="2")|(df.IRAN=="3")|(df.IRAN=="4")|(df.IRAN=="5")]

    # Indicateur de nationalité condensé (Français/Étranger) # Keep only French
    df = df[df.INATC=="1"]
    # Catégorie de population condensée
    df = df[df.CATPC=="0"]
    
    # restrict the age of the population which is studied :  

    df = df[(df.AGEREVQ=="020")|(df.AGEREVQ=="025")|(df.AGEREVQ=="030")|(df.AGEREVQ=="035")|(df.AGEREVQ=="040")|(df.AGEREVQ=="045")|(df.AGEREVQ=="050")|(df.AGEREVQ=="055")]
    
    df.to_pickle(f'./data_temp/migsample_{i}.pkl')
    
# First cleaning : Manage a concat. # probably need to add a year variable

df_13 = pd.read_pickle(f'./data_temp/migsample_2013.pkl')
df_14 = pd.read_pickle(f'./data_temp/migsample_2014.pkl')
df_15 = pd.read_pickle(f'./data_temp/migsample_2015.pkl')
df_16 = pd.read_pickle(f'./data_temp/migsample_2016.pkl')
df_17 = pd.read_pickle(f'./data_temp/migsample_2017.pkl')
df_18 = pd.read_pickle(f'./data_temp/migsample_2018.pkl')
df_19 = pd.read_pickle(f'./data_temp/migsample_2019.pkl')

dfs = [df_13,df_14,df_15,df_16,df_17,df_18,df_19]

 # List of your dataframes
df = pd.concat(dfs, axis=0)

os.remove(f'./data_temp/migsample_2013.pkl')
os.remove(f'./data_temp/migsample_2014.pkl')
os.remove(f'./data_temp/migsample_2015.pkl')
os.remove(f'./data_temp/migsample_2016.pkl')
os.remove(f'./data_temp/migsample_2017.pkl')
os.remove(f'./data_temp/migsample_2018.pkl')
os.remove(f'./data_temp/migsample_2019.pkl')

del df_13,df_14,df_15,df_16,df_17,df_18,df_19

#df.to_pickle(f'./data_temp/mig.pkl')
df.to_pickle(f'./data_temp/mig_sample.pkl')


In [4]:
df = pd.read_pickle(f'./data_temp/mig_sample.pkl')

In [5]:
# Should rename and drop columns we are not intrested in : 

cofint = [
# Mode de cohabitation   
'MOCO',
# Catégorie socioprofessionnelle de la personne de référence du ménage en 8 postes
'CSM',
# Indicateur urbain du lieu de résidence antérieure au 1er janvier de l'année précédente
'IRANUU',
# Lien à la personne de référence du ménage
'LPRM',
# Catégorie de population condensée
'CATPC',
# Diplôme le plus élevé
'DIPL',
# Type d'activité
'TACT',
# Ancienneté de recherche d'emploi #can drop because only active workers will be kept
##'RECH', 
    
# Type de logement
'TYPL',
# Département de naissance (si né en France)
'DNAI',
# Catégorie socioprofessionnelle en 8 postes
'CS1',
# Période regroupée d'achèvement de la construction de la maison ou de l'immeuble 
'ACHLR',
# Arrondissement municipal de résidence (Paris, Lyon et Marseille)
#'ARM',
# Nombre de personnes du ménage (regroupé) 
'NPERR',
# Sexe
'SEXE',
# Diplôme le plus élevé
'DIPL_15',
# Âge quinquennal en années révolues
'AGEREVQ',
# Condition d'emploi 
'EMPL',
# Type de construction 
'TYPC',
# Indicateur de résidence antérieure au 1er janvier de l'année précédente
'IRAN',
# Activité économique en 17 postes (NA - A17) 
'NA17',
# Département et commune du lieu de résidence
'COMMUNE',
# Indicateur Métropole ou DOM du lieu de résidence 
'METRODOM',
# Type de ménage regroupé (en 9 postes)
'TYPMR',
# Activité économique regroupée en 5 postes 
'NA5',
# Ancienneté d'emménagement condensée 
'ANEMC',
# Année
'year',
# Département et commune de résidence antérieure au 1er janvier de l'annéee précédente (arrondissement municipal pour Paris, Lyon, Marseille) 
'DCRAN',
# Indicateur du lieu de naissance 
'INAI',
# Indicateur de nationalité condensé (Français/Étranger)
'INATC',

# Âge regroupé de la personne de référence du ménage en 8 classes d'âge
'AGEMEN8',
# Statut d'occupation détaillé du logement 
'STOCD',
# Type d'activité de la personne de référence du ménage
'TACTM',
# Mode de transport principal le plus souvent utilisé pour aller travailler
'TRANS',
# Poids de l'individu
'IPONDI'
]

In [6]:
# Seconde cleaning : Maybe delete the data we are not intrested in : Foreigners, retired, young, Children, etc

In [7]:
"""
# Third cleaning : Here observation deletes & variable names change

#Replace all Z's, ZZ's and ZZZZZ's strings as NaN
df = df.replace('Z', -999)
df = df.replace('ZZ', -999)
df = df.replace('ZZZZZ', -999)
df = df.replace(np.nan, -999)

for name in df.columns:
    try:
        df.loc[:,name] = df.loc[:,name].astype(int)
    except ValueError:
        pass
"""

# Diplôme le plus élevé (post 2016 I think : To merge)
df['DIPL'] = df['DIPL'].replace('ZZ', -999)
df['DIPL'] = df['DIPL'].astype(float)
df.loc[df.loc[:,'DIPL']<13,'DIPL_15']='A'
df.loc[df.loc[:,'DIPL']==13,'DIPL_15']='B'
df.loc[(df.loc[:,'DIPL']==14) |(df.loc[:,'DIPL']==15),'DIPL_15']='C'
df.loc[(df.loc[:,'DIPL']==16) |(df.loc[:,'DIPL']==17)|(df.loc[:,'DIPL']==18)|(df.loc[:,'DIPL']==19),'DIPL_15']='D'
df.drop(columns={'DIPL'})
df['DIPL'] = df['DIPL'].astype(str).replace('-999','ZZ')
# Arrondissement municipal de résidence : Need to combine with 'commune'

# Sexe :
df['SEXE'] = df['SEXE'].replace({"2":"1","1":"0"})

# Activité économique en 17 postes (NA - A17) 

# Département et commune du lieu de résidence
df = df.rename(columns={'COMMUNE':'com_dest'})
# Indicateur Métropole ou DOM du lieu de résidence 
df = df[df.METRODOM=='M']

# Département et commune de résidence antérieure au 1er janvier de l'annéee précédente : # Need rename 
df = df.rename(columns={'DCRAN':'com_or'})

df['com_or'].astype(str)
df['com_dest'].astype(str)

#delete corsican towns
df = df[~df['com_or'].str.startswith(('2A','2B'))]
df = df[~df['com_dest'].str.startswith(('2A','2B'))]
# Don't want arrondissements so we replace by the code insee of the town

paris_arr = [*range(75101, 75121,1)]
lyon_arr = [*range(69381, 69390,1)]
marseille_arr = [*range(13201, 13217,1)]

df.loc[df['com_or'].astype(int).isin(paris_arr),'com_or'] = '75056'
df.loc[df['com_or'].astype(int).isin(lyon_arr),'com_or'] = '69123'
df.loc[df['com_or'].astype(int).isin(marseille_arr),'com_or'] = '13055'

df['com_or'].astype(str)
df['com_dest'].astype(str)

1          01001
4          01001
5          01001
13         01001
14         01001
           ...  
4999972    33463
4999981    33463
4999987    33463
4999997    33463
4999998    33463
Name: com_dest, Length: 10757957, dtype: object

In [8]:
###
### Better would be to create the geo info in python
###

# THEN NEED TO CLEAN THE DIFFERENT TOWN CHANGES 
geo_info = pd.read_csv(f"geo/geo_info.csv", sep=',',low_memory=False, dtype=str)

df_final = pd.merge(df, geo_info, how='left', left_on=['com_or','year'],right_on=['codgeo_old','year'])
#df = df_final.rename(columns={'':''}) ,
#df_final = df_final.drop(columns={'codegeo_new','codegeo'})
del df
df_final = pd.merge(df_final, geo_info, how='left', left_on=['com_dest','year'],right_on=['codgeo_old','year'],suffixes=('_or', '_des'))
#df_final.loc[df_final.loc[:,'codegeo_new']!=np.nan,'com_dest']= df_final['codegeo_new']
#df_final = df_final.drop(columns={'codegeo_new','codegeo'})
#df_final.to_pickle('cleaned_data/mig/migration.pkl.bz2', compression='bz2') 
df_final.to_pickle('cleaned_data/mig/migration_sample.pkl.bz2', compression='bz2') 

In [9]:
df_final

,com_dest,ARM,com_or,ACHLR,AGEMEN8,AGEREVQ,ANEMC,CATPC,CS1,CSM,...,dep_or,reg_or,codgeo_old_des,codgeo_des,libgeo_old_des,libgeo_des,ze2010_des,libze2010_des,dep_des,reg_des
0,01001,ZZZZZ,01001,5,25,040,2,0,3,3,...,01,84,01001,01001,Abergement-Clémenciat,L'Abergement-Clémenciat,8213,Villefranche-sur-Saône,01,84
1,01001,ZZZZZ,01001,5,55,055,3,0,3,3,...,01,84,01001,01001,Abergement-Clémenciat,L'Abergement-Clémenciat,8213,Villefranche-sur-Saône,01,84
2,01001,ZZZZZ,01001,5,55,055,3,0,5,3,...,01,84,01001,01001,Abergement-Clémenciat,L'Abergement-Clémenciat,8213,Villefranche-sur-Saône,01,84
3,01001,ZZZZZ,01001,5,40,045,3,0,6,6,...,01,84,01001,01001,Abergement-Clémenciat,L'Abergement-Clémenciat,8213,Villefranche-sur-Saône,01,84
4,01001,ZZZZZ,01001,5,25,030,2,0,4,3,...,01,84,01001,01001,Abergement-Clémenciat,L'Abergement-Clémenciat,8213,Villefranche-sur-Saône,01,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10765476,33463,ZZZZZ,33463,6,40,045,3,0,6,6,...,33,75,33463,33463,Saint-Pierre-d'Aurillac,Saint-Pierre-d'Aurillac,7204,Bordeaux,33,75
10765477,33463,ZZZZZ,33463,5,40,040,2,0,5,5,...,33,75,33463,33463,Saint-Pierre-d'Aurillac,Saint-Pierre-d'Aurillac,7204,Bordeaux,33,75
10765478,33463,ZZZZZ,33463,5,55,055,3,0,4,4,...,33,75,33463,33463,Saint-Pierre-d'Aurillac,Saint-Pierre-d'Aurillac,7204,Bordeaux,33,75
10765479,33463,ZZZZZ,33463,6,40,040,1,0,6,6,...,33,75,33463,33463,Saint-Pierre-d'Aurillac,Saint-Pierre-d'Aurillac,7204,Bordeaux,33,75


In [10]:
x  = df_final.loc[(df_final['ze2010_des'].isna())|(df_final['ze2010_or'].isna()),['year','com_dest','com_or','codgeo_old_des','codgeo_des','codgeo_old_or','codgeo_or']]
x.to_csv(f'./data_temp/unmatched_communes.pkl')

In [11]:
x

,year,com_dest,com_or,codgeo_old_des,codgeo_des,codgeo_old_or,codgeo_or
671317,2013,14513,14118,NaN,NaN,14118,14118
671318,2013,14513,14118,NaN,NaN,14118,14118
671319,2013,14513,14513,NaN,NaN,NaN,NaN
671320,2013,14513,14513,NaN,NaN,NaN,NaN
671321,2013,14513,14513,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
10248637,2019,27058,27058,NaN,NaN,NaN,NaN
10248638,2019,27058,27058,NaN,NaN,NaN,NaN
10248639,2019,27058,27517,NaN,NaN,27517,27517
10248640,2019,27058,27517,NaN,NaN,27517,27517


In [12]:
x.loc[x.codgeo_des.isna(), 'com_dest'].unique()

array(['14513', '14472', '27058'], dtype=object)

In [13]:
x.loc[x.codgeo_or.isna(), 'com_or'].unique()

array(['14513', '76095', '44060', '14472', '27058'], dtype=object)